In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1743275365915_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
%%capture
%%bash
wget -O - https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz | aws s3 cp - s3://redfin-source-raw-data-emr/city_market_tracker.tsv000.gz

In [5]:
spark = (
    SparkSession
    .builder
    .appName('Redfin-BigData-Analysis')
    .getOrCreate()
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
redfin_data = spark.read.csv('s3://redfin-source-raw-data-emr/city_market_tracker.tsv000.gz', 
                              sep='\t', header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
redfin_data.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------+---------------+-----------+--------------+--------+----------------------+------------+--------+---------+----------+--------------------+----------------+-----------------+---------------------+---------------------+-----------------+---------------------+---------------------+------------------+--------------------+-------------------+------------------+--------------------+--------------------+----------+--------------------+--------------------+-------------+--------------------+--------------------+------------+-------------------+-------------------+---------+--------------------+--------------------+----------------+--------------------+--------------------+----------+--------------+--------------+------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-----------------------+---------------------------+-----------------------

In [9]:
redfin_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- period_begin: date (nullable = true)
 |-- period_end: date (nullable = true)
 |-- period_duration: integer (nullable = true)
 |-- region_type: string (nullable = true)
 |-- region_type_id: integer (nullable = true)
 |-- table_id: integer (nullable = true)
 |-- is_seasonally_adjusted: string (nullable = true)
 |-- region: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- property_type: string (nullable = true)
 |-- property_type_id: integer (nullable = true)
 |-- median_sale_price: double (nullable = true)
 |-- median_sale_price_mom: double (nullable = true)
 |-- median_sale_price_yoy: double (nullable = true)
 |-- median_list_price: double (nullable = true)
 |-- median_list_price_mom: double (nullable = true)
 |-- median_list_price_yoy: double (nullable = true)
 |-- median_ppsf: double (nullable = true)
 |-- median_ppsf_mom: double (nullable = true)
 |-- median_ppsf_yoy: double (nul

In [10]:
# Total Records

redfin_data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5779946

In [11]:
df_redfin = redfin_data.select('period_end','period_duration', 'city', 'state', 'property_type',
    'median_sale_price', 'median_ppsf', 'homes_sold', 'inventory', 'months_of_supply', 'median_dom', 'sold_above_list', 'last_updated')
df_redfin.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+--------+---------+--------------------+-----------------+------------------+----------+---------+----------------+----------+-------------------+-------------------+
|period_end|period_duration|    city|    state|       property_type|median_sale_price|       median_ppsf|homes_sold|inventory|months_of_supply|median_dom|    sold_above_list|       last_updated|
+----------+---------------+--------+---------+--------------------+-----------------+------------------+----------+---------+----------------+----------+-------------------+-------------------+
|2016-09-30|             30|Waukegan| Illinois|Multi-Family (2-4...|         100000.0| 47.20620366179577|         4|       20|             5.0|        55|               0.25|2025-03-10 14:20:36|
|2019-03-31|             30|Zumbrota|Minnesota|Single Family Res...|         233500.0|142.08238592397709|         4|       13|             3.3|        80|               0.25|2025-03-10 14:20:36|
|2015-08-31|             

In [18]:
from pyspark.sql.functions import isnull

df_redfin = df_redfin.na.drop()

print('Total Rows:', df_redfin.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total Rows: 5214363

In [19]:
from pyspark.sql.functions import avg, col

df_redfin.groupBy('state') \
    .agg(avg(col('median_sale_price')).alias('Avg_Median_Sale_Price')) \
    .orderBy(col('Avg_Median_Sale_Price').desc()) \
    .show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+---------------------+
|        state|Avg_Median_Sale_Price|
+-------------+---------------------+
|       Hawaii|    741692.0023227786|
|      Wyoming|    693313.1335917313|
|   California|     666470.179381025|
|     Columbia|    645509.8759493671|
|     Colorado|   521543.37136866484|
|Massachusetts|    456342.8571943437|
|  Connecticut|    434241.7186949657|
|   Washington|   433656.33993819304|
|       Nevada|    430992.2907547092|
|   New Jersey|    408715.8338951671|
|         Utah|   393817.37809755874|
| Rhode Island|    385144.0207027047|
|      Montana|   383349.92446642375|
|     Virginia|    372218.5157306554|
|     New York|   351016.09363502526|
|     Maryland|   349141.28027003625|
|       Oregon|   341514.35884725256|
|   New Mexico|    326561.0066881403|
|      Florida|    325276.8669064219|
|        Idaho|    322185.6420338353|
+-------------+---------------------+
only showing top 20 rows


### 🔹 1. **Trend in Median Sale Price by State Over Time**
> For each state, calculate the average median sale price **per year**, and order by year and descending price within each year.



In [20]:
df_redfin.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- period_end: date (nullable = true)
 |-- period_duration: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- property_type: string (nullable = true)
 |-- median_sale_price: double (nullable = true)
 |-- median_ppsf: double (nullable = true)
 |-- homes_sold: integer (nullable = true)
 |-- inventory: integer (nullable = true)
 |-- months_of_supply: double (nullable = true)
 |-- median_dom: integer (nullable = true)
 |-- sold_above_list: double (nullable = true)
 |-- last_updated: timestamp (nullable = true)

In [21]:
from pyspark.sql.functions import *

df_redfin = df_redfin.withColumn('Period_End_Year' , year(col('period_end')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df_redfin.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+--------+---------+--------------------+-----------------+------------------+----------+---------+----------------+----------+-------------------+-------------------+---------------+
|period_end|period_duration|    city|    state|       property_type|median_sale_price|       median_ppsf|homes_sold|inventory|months_of_supply|median_dom|    sold_above_list|       last_updated|Period_End_Year|
+----------+---------------+--------+---------+--------------------+-----------------+------------------+----------+---------+----------------+----------+-------------------+-------------------+---------------+
|2016-09-30|             30|Waukegan| Illinois|Multi-Family (2-4...|         100000.0| 47.20620366179577|         4|       20|             5.0|        55|               0.25|2025-03-10 14:20:36|           2016|
|2019-03-31|             30|Zumbrota|Minnesota|Single Family Res...|         233500.0|142.08238592397709|         4|       13|             3.3|        80|  

In [25]:
df_redfin.groupBy('Period_End_Year', 'state').agg(avg(col('median_sale_price')).alias('Avg_Median_Sale_Price')).orderBy(col('Period_End_Year').desc(), col('Avg_Median_Sale_Price').desc()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-------------+---------------------+
|Period_End_Year|        state|Avg_Median_Sale_Price|
+---------------+-------------+---------------------+
|           2025|      Wyoming|            1248825.0|
|           2025|       Hawaii|   1103347.5338983051|
|           2025|   California|    934876.8782513005|
|           2025|     Colorado|    819017.2590132827|
|           2025|     Columbia|             783150.0|
|           2025| Rhode Island|    730738.0246710526|
|           2025|  Connecticut|    695182.4123252858|
|           2025|Massachusetts|    693781.5480811645|
|           2025|      Montana|    679236.4177777778|
|           2025|   Washington|    672967.3835145965|
|           2025|   New Jersey|    649736.1395899054|
|           2025|       Nevada|        643174.828125|
|           2025|         Utah|    638767.5775434243|
|           2025|      Florida|    558387.6501511979|
|           2025|     New York|    547379.9712338262|
|           2025|        Ida

### 🔹 2. **Top 3 Cities per State by Average Price per Square Foot**
> For each state, find the top 3 cities with the highest **average median_ppsf** (price per square foot).


In [31]:
from pyspark.sql.functions import avg, round, col, dense_rank
from pyspark.sql.window import Window

df_avg_ppsf = df_redfin.groupBy('state', 'city') \
    .agg(round(avg(col('median_ppsf')), 2).alias('avg_median_ppsf'))

window_spec = Window.partitionBy('state').orderBy(col('avg_median_ppsf').desc())

df_top3 = df_avg_ppsf.withColumn('rank', dense_rank().over(window_spec)) \
    .filter(col('rank') <= 3)


df_top3.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------------+---------------+----+
|      state|           city|avg_median_ppsf|rank|
+-----------+---------------+---------------+----+
|    Alabama|   Orange Beach|         298.99|   1|
|    Alabama| Dauphin Island|         277.35|   2|
|    Alabama|    Point Clear|         240.45|   3|
|     Alaska|         Salcha|        1053.95|   1|
|     Alaska|     North Pole|         552.78|   2|
|     Alaska|      Ketchikan|         429.57|   3|
|    Arizona|       Show Low|         913.82|   1|
|    Arizona|Paradise Valley|         454.71|   2|
|    Arizona|  Martinez Lake|         426.31|   3|
|   Arkansas|  Lake Hamilton|         178.52|   1|
|   Arkansas|Cammack Village|         169.24|   2|
|   Arkansas|    Bentonville|         157.58|   3|
| California|    West Athens|        5158.64|   1|
| California|    Terra Bella|        3403.23|   2|
| California|San Luis Obispo|        3346.55|   3|
|   Colorado|   Jackson Lake|        3189.19|   1|
|   Colorado|      Fruitvale|  

### 🔹 3. **Days on Market Analysis**
> Find the states where the **average median days on market (median_dom)** has been **increasing over the last 3 years**.


In [38]:
from pyspark.sql.functions import avg, round, col

df_2023 = df_redfin.filter('Period_End_Year = 2023') \
    .groupBy('state') \
    .agg(round(avg(col('median_dom')), 2).alias('average_dom_2023'))

df_2024 = df_redfin.filter('Period_End_Year = 2024') \
    .groupBy('state') \
    .agg(avg(col('median_dom')).alias('average_dom_2024'))

df_2025 = df_redfin.filter('Period_End_Year = 2025') \
    .groupBy('state') \
    .agg(avg(col('median_dom')).alias('average_dom_2025'))

res_df = (
    df_2023.alias('d1')
    .join(df_2024.alias('d2'), col('d1.state') == col('d2.state'), how='inner')
    .join(df_2025.alias('d3'), col('d1.state') == col('d3.state'), how='inner')
    .select(
        col('d1.state'),
        col('d1.average_dom_2023'),
        col('d2.average_dom_2024'),
        col('d3.average_dom_2025')
    )
    .filter(
        (col('average_dom_2025') > col('average_dom_2024')) &
        (col('average_dom_2024') > col('average_dom_2023'))
    )
)

res_df.show(10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------------+------------------+------------------+
|        state|average_dom_2023|  average_dom_2024|  average_dom_2025|
+-------------+----------------+------------------+------------------+
|    Minnesota|           46.78| 58.58201659656701| 68.44620253164557|
|     Oklahoma|           54.07| 64.47876356772062|  85.9400299850075|
|     Maryland|           38.54|39.322119732320495| 53.78974956319161|
|Massachusetts|           33.41|34.271618107873856|47.617556241729154|
|     Kentucky|           51.09| 51.73813199536858| 78.59245742092457|
|  Connecticut|            48.5| 49.41467211519751| 67.63532401524778|
|       Kansas|           47.76|           52.5468| 73.58445040214477|
|         Iowa|           66.73|  70.1921630714427| 72.44923504867872|
|      Florida|           57.24| 73.47640282422891|  90.6624796464294|
|       Hawaii|          153.42|156.26662580447442|196.44821092278718|
+-------------+----------------+------------------+------------------+
only s


### 🔹 4. **Months of Supply Category**
> Create a new column categorizing the `months_of_supply` into:
- `"Low"` if < 2
- `"Medium"` if between 2 and 6
- `"High"` if > 6


In [39]:
from pyspark.sql.functions import when, col

df_redfin = df_redfin.withColumn(
    'months_of_supply_category',
    when(col('months_of_supply') < 2, 'Low')
    .when(col('months_of_supply') < 6, 'Medium')
    .otherwise('High')
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
s3_bucket = 's3://redfin-transformed-data-emr/redfin-data.parquet'
df_redfin.write.mode('overwrite').parquet(s3_bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…